# Chap 4 : Word2Vec 속도 개선

## 다음 두 가지를 진행
- 입력층 원핫 벡터와 가중치 $W_{in}$ 간의 곱 계산에서 발생하는 문제 => Embedding 계층 도입
- 은닉층과 가중치 $W_{out}$ 간의 곱, Softmax 계층 계산에서 발생하는 문제 => Negative Sampling 도입

![ㅁㄴㅇㄹ](img/chap_04/01.png)

## 1. Em|bedding 계층
- 단어를 원핫으로의 변환과 MatMul 계층은 사실상 $W_{in}$의 특정 행을 추출하는 작업
- 단어 ID에 해당하는 행을 추출하는 계층을 통해 해결

![ㅁㄴㅇㄹ](img/chap_04/02.png)

### 1-1. Embedding 계층 구현

In [1]:
import numpy as np

W = np.arange(21).reshape(7,3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [2]:
W[2]

array([6, 7, 8])

#### 미니배치를 고려한 행 추출

In [3]:
W[[0, 1, 3]]
# idx = np.array([0, 1, 3])
# W[idx]

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 9, 10, 11]])

### 1-2. Embedding 계층의 forward() 구현

In [4]:
class Embedding:
    def __init__(self, W):
        self.params = [W]  # 하나밖에 없지만 레이어 생성 규칙이라 배열
        self.grads = [np.zeros_like(W)]  # 하나밖에 없지만 레이어 생성 규칙이라 배열
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

In [5]:
W = np.random.randn(4, 4)
W

array([[ 1.65039271,  0.69025171,  0.52157334, -0.30134532],
       [-1.44200684,  0.82186024,  1.15116599,  1.25567187],
       [ 1.17411856,  0.34702447, -0.61113509, -0.04681875],
       [ 2.82794695,  1.73938041,  0.5528964 ,  0.45970765]])

In [6]:
embdding = Embedding(W)

embdding.forward([0, 2])

array([[ 1.65039271,  0.69025171,  0.52157334, -0.30134532],
       [ 1.17411856,  0.34702447, -0.61113509, -0.04681875]])

### 1-3. Embedding 계층의 backward() 구현
- 앞 층으로부터 전해진 기울기를 가중치 기울기 $dW$의 특정 행에 설정

![ㅁㄴㅇㄹ](img/chap_04/03.png)

In [7]:
class Embedding:
    def __init__(self, W):
        self.params = [W]  # 하나밖에 없지만 레이어 생성 규칙이라 배열
        self.grads = [np.zeros_like(W)]  # 하나밖에 없지만 레이어 생성 규칙이라 배열
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0  # 기울기의 형상을 유지한 상태로 0으로 초기화
        dW[self.idx] = dout
        print("id(self.grads):", id(self.grads))
        print("type(self.grads):", type(self.grads))
        print(self.grads)
        print()
        print("id(dW):", id(dW))
        print("type(dW):", type(dW))
        print(dW)
        return None

In [8]:
W = np.random.randn(4, 4)
W
embdding = Embedding(W)

out = embdding.forward([0, 2])
embdding.backward(out)

id(self.grads): 2575983732744
type(self.grads): <class 'list'>
[array([[ 0.21653844, -0.79088163,  0.10271234,  0.35839855],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.43173284, -1.36803628,  0.80290825,  0.11675985],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])]

id(dW): 2575983858912
type(dW): <class 'numpy.ndarray'>
[[ 0.21653844 -0.79088163  0.10271234  0.35839855]
 [ 0.          0.          0.          0.        ]
 [ 0.43173284 -1.36803628  0.80290825  0.11675985]
 [ 0.          0.          0.          0.        ]]


### 1-4. Backward() 구현의 문제 및 해결
- idx의 원소가 중복될 경우, $dW$를 덮어쓰는 문제 발생
- 이를 해결하기 위해 해당 행의 $dW$가 존재한다면, 값을 더함(Repeat 노드의 역전파)

![ㅁㄴㅇㄹ](img/chap_04/04.png)

In [14]:
class Embedding:
    def __init__(self, W):
        self.params = [W]  # 하나밖에 없지만 레이어 생성 규칙이라 배열
        self.grads = [np.zeros_like(W)]  # 하나밖에 없지만 레이어 생성 규칙이라 배열
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0  # 기울기의 형상을 유지한 상태로 0으로 초기화
        
        for i, word_id in enumerate(self.idx):
            dW[word_id] += dout[i]  # Repeat 노드의 역전파 = Sum 노드
            
        # np.add.at(dW, self.idx, dout)
        
        print("id(self.grads):", id(self.grads))
        print("type(self.grads):", type(self.grads))
        print(self.grads)
        print()
        print("id(dW):", id(dW))
        print("type(dW):", type(dW))
        print(dW)
        return None

In [15]:
W = np.random.randn(4, 4)
W
embdding = Embedding(W)

out = embdding.forward([0, 2])
embdding.backward(out)

id(self.grads): 2575983875208
type(self.grads): <class 'list'>
[array([[ 1.59015297, -1.50133133,  0.70998179,  0.09865617],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 1.17473646, -1.54453985, -1.65016348, -0.49054014],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])]

id(dW): 2575983905088
type(dW): <class 'numpy.ndarray'>
[[ 1.59015297 -1.50133133  0.70998179  0.09865617]
 [ 0.          0.          0.          0.        ]
 [ 1.17473646 -1.54453985 -1.65016348 -0.49054014]
 [ 0.          0.          0.          0.        ]]


## 2. Nagative sampling
- Embedding 적용후 은닉층 이후 계산이 오래 걸림
    - 은닉층 뉴런과 가중치 행렬 $W_{out}$의 곱
    - Softmax 계층 계산
- 다중 분류 => 이진분류로 근사

![ㅁㄴㅇㄹ](img/chap_04/05.png)
![ㅁㄴㅇㄹ](img/chap_04/06.png)
![ㅁㄴㅇㄹ](img/chap_04/07.png)

### 2-1. Sigmoid with Loss
- 이진 분류 문제로 변환하기 때문에 sigmoid와 Cross Entropy Error 사용

![image.png](img/chap_04/08.png)

### 2-2. 이진분류 구현
- 기존 다중분류를 수행하는 모델
![image.png](img/chap_04/09.png)

- 이진분류를 수행하는 모델
![image.png](img/chap_04/10.png)

- 은닉층 이후를 Embedding 계층과 dot 연산을 합친 Embedding Dot 계층을 이용
![image.png](img/chap_04/11.png)

#### EmbeddingDot 클래스

In [16]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)  # 기존 임베딩 계층과 동일
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None  # 순전파 계산 결과 임시 저장용
        
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)  # idx는 미니배치를 위해 리스트
        out = np.sum(target_W * h, axis=1)  # 각 벡터의 내적을 쉽게 구하는 트릭
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

#### SigmoidWithLoss 클래스

- Cross Entropy Error

$L = -(t\log{y} + (1 - t)\log(1-y))$

In [198]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 정답 데이터가 원핫 벡터일 경우 정답 레이블 인덱스로 변환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]

    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size


class SigmoidWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.loss = None
        self.y = None  # sigmoid의 출력
        self.t = None  # 정답 데이터

    def forward(self, x, t):
        self.t = t
        self.y = 1 / (1 + np.exp(-x))  # 시그모이드

        self.loss = cross_entropy_error(np.c_[1 - self.y, self.y], self.t)

        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]

        dx = (self.y - self.t) * dout / batch_size
        return dx

### 2-3. Negative sampling
- 다중 분류 문제를 이진 분류 문제로 변환하였지만, 긍정적인 예만 학습
- 부정적인 예도 학습하여 모델의 분류 능력을 향상시킬 필요 있음
- 모든 부정적 예를 다 학습하는 것은 힘들고 이진 분류를 하는 이점이 없음
- 적은 수의 부정적인 예를 샘플링하여 같이 학습

![image.png](img/chap_04/12.png)
![image.png](img/chap_04/13.png)

#### 샘플링 기법
- 말뭉치에서 각 단어의 빈도수를 구하여 확률 분포로 나타냄
- 말뭉치에서 자주 등장하는 단어를 많이 추출
- 드물게 등장하는 단어를 적게 추출

In [172]:
for i in range(5):
    print(np.random.choice(10))

3
9
1
8
7


In [181]:
words = ['you', 'say', 'goodbye', 'I', 'hello', '.']
np.random.choice(words)

'say'

In [193]:
np.random.choice(words, size=3)

array(['say', 'say', 'goodbye'], dtype='<U7')

In [197]:
np.random.choice(words, size=3, replace=False)  # 중복 없음

array(['.', 'say', 'hello'], dtype='<U7')

In [206]:
data = np.random.randn(6)
data

array([-0.6939333 , -0.56233325,  0.68469457,  0.95888722,  0.63085552,
       -0.20714661])

In [207]:
def softmax(x):
    if x.ndim == 2:
        x = x - x.max(axis=1, keepdims=True)
        x = np.exp(x)
        x /= x.sum(axis=1, keepdims=True)
    elif x.ndim == 1:
        x = x - np.max(x)
        x = np.exp(x) / np.sum(np.exp(x))

    return x

In [211]:
p = softmax(data)
p

array([0.05980768, 0.06821975, 0.23740367, 0.31229698, 0.22496007,
       0.09731185])

In [217]:
np.random.choice(words, p=p)

'you'

#### word2vec 네거티브 샘플링은 확률분포에 0.75를 곱할 것을 권고
- 출현이 낮은 단어를 버리지 않기 위해 사용
- 확률이 낮은 단어의 확률을 좀 높여, 샘플링될 확률을 올려줌

$P'(w_i) = \frac{P(w_i)^{0.75}}{\sum_{j}^{n}{P(w_j)^{0.75}}}$

In [220]:
new_p = np.power(p, 0.75)
new_p /= np.sum(new_p)

[(i, j) for i, j in zip(p, new_p)]

[(0.05980767962382291, 0.07992460237183979),
 (0.06821975127705387, 0.08821559507997324),
 (0.2374036723322005, 0.2247648236380674),
 (0.3122969820199227, 0.2760823289193763),
 (0.22496006518151723, 0.2158697761778994),
 (0.09731184956548286, 0.11514287381284394)]

### 2-4 부정적 예를 샘플링하는 클래스

In [226]:
import collections

class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        GPU = False
        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        else:
            # GPU(cupy）로 계산할 때는 속도를 우선한다.
            # 부정적 예에 타깃이 포함될 수 있다.
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample

In [227]:
corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1, 3, 0])
negative_sample = sampler.get_negative_sample(target)
negative_sample

array([[4, 2],
       [1, 0],
       [2, 3]])

### 2-5 NegativeSampling 구현